<a href="https://colab.research.google.com/github/Shailja12326646/AI_AGENTS/blob/main/price_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import pandas as pd

In [20]:
import zipfile
import os
zip_path = "/content/68e8d1d70b66d_student_resource.zip"

extract_dir = "/content/dataset"
os.makedirs(extract_dir, exist_ok=True)
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("✅ Files extracted to:", extract_dir)


✅ Files extracted to: /content/dataset


In [21]:
df=pd.read_csv("/content/dataset/student_resource/dataset/train.csv")

In [22]:
df.columns = ['id', 'text_block', 'image_url','price']


In [23]:
import re

def extract_features(text):
    # Initialize dictionary
    data = {
        "item_name": None,
        "bullets": [],
        "value": None,
        "unit": None
    }

    # Extract item name
    item = re.search(r"Item Name:\s*(.*)", text)
    if item:
        data["item_name"] = item.group(1).strip()

    # Extract all bullet points
    bullets = re.findall(r"Bullet Point \d+:\s*(.*)", text)
    data["bullets"] = bullets

    # Extract value and unit
    value = re.search(r"Value:\s*([\d.]+)", text)
    if value:
        data["value"] = float(value.group(1))

    unit = re.search(r"Unit:\s*([A-Za-z\s]+)", text)
    if unit:
        data["unit"] = unit.group(1).strip()

    return data


In [24]:
features = df['text_block'].apply(extract_features)
feature_df = pd.DataFrame(features.tolist())

# Combine with original data
df = pd.concat([df, feature_df], axis=1)

df.head(10)


,id,text_block,image_url,price,item_name,bullets,value,unit
0,33127,"Item Name: La Victoria Green Taco Sauce Mild, ...",https://m.media-amazon.com/images/I/51mo8htwTH...,4.89,"La Victoria Green Taco Sauce Mild, 12 Ounce (P...",[],72.00,Fl Oz
1,198967,"Item Name: Salerno Cookies, The Original Butte...",https://m.media-amazon.com/images/I/71YtriIHAA...,13.12,"Salerno Cookies, The Original Butter Cookies, ...",[Original Butter Cookies: Classic butter cooki...,32.00,Ounce
2,261251,"Item Name: Bear Creek Hearty Soup Bowl, Creamy...",https://m.media-amazon.com/images/I/51+PFEe-w-...,1.97,"Bear Creek Hearty Soup Bowl, Creamy Chicken wi...",[Loaded with hearty long grain wild rice and v...,11.40,Ounce
3,55858,Item Name: Judee’s Blue Cheese Powder 11.25 oz...,https://m.media-amazon.com/images/I/41mu0HAToD...,30.34,Judee’s Blue Cheese Powder 11.25 oz - Gluten-F...,"[Add to your favorite appetizers, dips & sprea...",11.25,Ounce
4,292686,"Item Name: kedem Sherry Cooking Wine, 12.7 Oun...",https://m.media-amazon.com/images/I/41sA037+Qv...,66.49,"kedem Sherry Cooking Wine, 12.7 Ounce - 12 per...",[],12.00,Count
5,9259,"Item Name: Member's Mark Member's Mark, Basil,...",https://m.media-amazon.com/images/I/81nw0HXpCR...,18.50,"Member's Mark Member's Mark, Basil, 6.25 oz","[Green Herb, Italian Staple, Great mixed with ...",6.25,ounce
6,191846,Item Name: Goya Foods Sazonador Total Seasonin...,https://m.media-amazon.com/images/I/61dH2Ebkt0...,5.99,"Goya Foods Sazonador Total Seasoning, 30 Ounce...",[SAZONADOR TOTAL | Enhance the natural flavors...,180.00,Ounce
7,222007,Item Name: VineCo Original Series Chilean Sauv...,https://m.media-amazon.com/images/I/71JllaFpxM...,94.00,VineCo Original Series Chilean Sauvignon Blanc...,[Chilean Sauvignon Blanc Wine Kit - VineCo Ori...,1.00,Count
8,37614,Item Name: NATURES PATH CEREAL FLK MULTIGRAIN ...,https://m.media-amazon.com/images/I/21O9RftI2v...,35.74,"NATURES PATH CEREAL FLK MULTIGRAIN ORG ECO, 32...",[],192.00,Fl Oz
9,238044,Item Name: Mrs. Miller's Seedless Black Raspbe...,https://m.media-amazon.com/images/I/41miQk+RkJ...,31.80,Mrs. Miller's Seedless Black Raspberry Jam 9 O...,[Homemade Seedless Black Raspberry Jam made fr...,9.00,Ounce


In [25]:
df["all_text"] = df["item_name"].fillna('') + " " + df["bullets"].apply(lambda x: " ".join(x))


In [27]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer

def clean_text(text):
    text = text.lower()
    text = re.sub(r"[^a-z0-9\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

df["clean_text"] = df["all_text"].apply(clean_text)


In [28]:
df.head()

,id,text_block,image_url,price,item_name,bullets,value,unit,all_text,clean_text
0,33127,"Item Name: La Victoria Green Taco Sauce Mild, ...",https://m.media-amazon.com/images/I/51mo8htwTH...,4.89,"La Victoria Green Taco Sauce Mild, 12 Ounce (P...",[],72.00,Fl Oz,"La Victoria Green Taco Sauce Mild, 12 Ounce (P...",la victoria green taco sauce mild 12 ounce pac...
1,198967,"Item Name: Salerno Cookies, The Original Butte...",https://m.media-amazon.com/images/I/71YtriIHAA...,13.12,"Salerno Cookies, The Original Butter Cookies, ...",[Original Butter Cookies: Classic butter cooki...,32.00,Ounce,"Salerno Cookies, The Original Butter Cookies, ...",salerno cookies the original butter cookies 8 ...
2,261251,"Item Name: Bear Creek Hearty Soup Bowl, Creamy...",https://m.media-amazon.com/images/I/51+PFEe-w-...,1.97,"Bear Creek Hearty Soup Bowl, Creamy Chicken wi...",[Loaded with hearty long grain wild rice and v...,11.40,Ounce,"Bear Creek Hearty Soup Bowl, Creamy Chicken wi...",bear creek hearty soup bowl creamy chicken wit...
3,55858,Item Name: Judee’s Blue Cheese Powder 11.25 oz...,https://m.media-amazon.com/images/I/41mu0HAToD...,30.34,Judee’s Blue Cheese Powder 11.25 oz - Gluten-F...,"[Add to your favorite appetizers, dips & sprea...",11.25,Ounce,Judee’s Blue Cheese Powder 11.25 oz - Gluten-F...,judee s blue cheese powder 11 25 oz gluten fre...
4,292686,"Item Name: kedem Sherry Cooking Wine, 12.7 Oun...",https://m.media-amazon.com/images/I/41sA037+Qv...,66.49,"kedem Sherry Cooking Wine, 12.7 Ounce - 12 per...",[],12.00,Count,"kedem Sherry Cooking Wine, 12.7 Ounce - 12 per...",kedem sherry cooking wine 12 7 ounce 12 per case


In [29]:
vectorizer = TfidfVectorizer(max_features=500)
X_tfidf = vectorizer.fit_transform(df["clean_text"])

print("Shape of TF-IDF matrix:", X_tfidf.shape)


Shape of TF-IDF matrix: (75000, 500)


In [30]:
import re

def extract_value(text):
    if isinstance(text, str):
        match = re.search(r"Value:\s*([\d.]+)", text)
        if match:
            return float(match.group(1))
    return None  # return None if not found or invalid text


In [32]:
df["extracted_value"] = df["text_block"].apply(extract_value)


In [33]:
df[["text_block", "extracted_value"]].head(5)


,text_block,extracted_value
0,"Item Name: La Victoria Green Taco Sauce Mild, ...",72.00
1,"Item Name: Salerno Cookies, The Original Butte...",32.00
2,"Item Name: Bear Creek Hearty Soup Bowl, Creamy...",11.40
3,Item Name: Judee’s Blue Cheese Powder 11.25 oz...,11.25
4,"Item Name: kedem Sherry Cooking Wine, 12.7 Oun...",12.00


In [34]:
df.head()

,id,text_block,image_url,price,item_name,bullets,value,unit,all_text,clean_text,extracted_value
0,33127,"Item Name: La Victoria Green Taco Sauce Mild, ...",https://m.media-amazon.com/images/I/51mo8htwTH...,4.89,"La Victoria Green Taco Sauce Mild, 12 Ounce (P...",[],72.00,Fl Oz,"La Victoria Green Taco Sauce Mild, 12 Ounce (P...",la victoria green taco sauce mild 12 ounce pac...,72.00
1,198967,"Item Name: Salerno Cookies, The Original Butte...",https://m.media-amazon.com/images/I/71YtriIHAA...,13.12,"Salerno Cookies, The Original Butter Cookies, ...",[Original Butter Cookies: Classic butter cooki...,32.00,Ounce,"Salerno Cookies, The Original Butter Cookies, ...",salerno cookies the original butter cookies 8 ...,32.00
2,261251,"Item Name: Bear Creek Hearty Soup Bowl, Creamy...",https://m.media-amazon.com/images/I/51+PFEe-w-...,1.97,"Bear Creek Hearty Soup Bowl, Creamy Chicken wi...",[Loaded with hearty long grain wild rice and v...,11.40,Ounce,"Bear Creek Hearty Soup Bowl, Creamy Chicken wi...",bear creek hearty soup bowl creamy chicken wit...,11.40
3,55858,Item Name: Judee’s Blue Cheese Powder 11.25 oz...,https://m.media-amazon.com/images/I/41mu0HAToD...,30.34,Judee’s Blue Cheese Powder 11.25 oz - Gluten-F...,"[Add to your favorite appetizers, dips & sprea...",11.25,Ounce,Judee’s Blue Cheese Powder 11.25 oz - Gluten-F...,judee s blue cheese powder 11 25 oz gluten fre...,11.25
4,292686,"Item Name: kedem Sherry Cooking Wine, 12.7 Oun...",https://m.media-amazon.com/images/I/41sA037+Qv...,66.49,"kedem Sherry Cooking Wine, 12.7 Ounce - 12 per...",[],12.00,Count,"kedem Sherry Cooking Wine, 12.7 Ounce - 12 per...",kedem sherry cooking wine 12 7 ounce 12 per case,12.00


In [35]:
df["item_name"] = df["item_name"].str.replace("Item Name:", "", regex=False).str.strip()


In [36]:
import re

def extract_brand(name):
    if not isinstance(name, str):
        return None

    # Try to capture the first 1–2 capitalized words
    match = re.match(r"([A-Z][a-zA-Z]*(?:\s+[A-Z][a-zA-Z]*)?)", name)
    if match:
        return match.group(1).strip()
    return None


In [37]:
df["brand_name"] = df["item_name"].apply(extract_brand)


In [39]:
df.head(10)

,id,text_block,image_url,price,item_name,bullets,value,unit,all_text,clean_text,extracted_value,brand_name
0,33127,"Item Name: La Victoria Green Taco Sauce Mild, ...",https://m.media-amazon.com/images/I/51mo8htwTH...,4.89,"La Victoria Green Taco Sauce Mild, 12 Ounce (P...",[],72.00,Fl Oz,"La Victoria Green Taco Sauce Mild, 12 Ounce (P...",la victoria green taco sauce mild 12 ounce pac...,72.00,La Victoria
1,198967,"Item Name: Salerno Cookies, The Original Butte...",https://m.media-amazon.com/images/I/71YtriIHAA...,13.12,"Salerno Cookies, The Original Butter Cookies, ...",[Original Butter Cookies: Classic butter cooki...,32.00,Ounce,"Salerno Cookies, The Original Butter Cookies, ...",salerno cookies the original butter cookies 8 ...,32.00,Salerno Cookies
2,261251,"Item Name: Bear Creek Hearty Soup Bowl, Creamy...",https://m.media-amazon.com/images/I/51+PFEe-w-...,1.97,"Bear Creek Hearty Soup Bowl, Creamy Chicken wi...",[Loaded with hearty long grain wild rice and v...,11.40,Ounce,"Bear Creek Hearty Soup Bowl, Creamy Chicken wi...",bear creek hearty soup bowl creamy chicken wit...,11.40,Bear Creek
3,55858,Item Name: Judee’s Blue Cheese Powder 11.25 oz...,https://m.media-amazon.com/images/I/41mu0HAToD...,30.34,Judee’s Blue Cheese Powder 11.25 oz - Gluten-F...,"[Add to your favorite appetizers, dips & sprea...",11.25,Ounce,Judee’s Blue Cheese Powder 11.25 oz - Gluten-F...,judee s blue cheese powder 11 25 oz gluten fre...,11.25,Judee
4,292686,"Item Name: kedem Sherry Cooking Wine, 12.7 Oun...",https://m.media-amazon.com/images/I/41sA037+Qv...,66.49,"kedem Sherry Cooking Wine, 12.7 Ounce - 12 per...",[],12.00,Count,"kedem Sherry Cooking Wine, 12.7 Ounce - 12 per...",kedem sherry cooking wine 12 7 ounce 12 per case,12.00,None
5,9259,"Item Name: Member's Mark Member's Mark, Basil,...",https://m.media-amazon.com/images/I/81nw0HXpCR...,18.50,"Member's Mark Member's Mark, Basil, 6.25 oz","[Green Herb, Italian Staple, Great mixed with ...",6.25,ounce,"Member's Mark Member's Mark, Basil, 6.25 oz Gr...",member s mark member s mark basil 6 25 oz gree...,6.25,Member
6,191846,Item Name: Goya Foods Sazonador Total Seasonin...,https://m.media-amazon.com/images/I/61dH2Ebkt0...,5.99,"Goya Foods Sazonador Total Seasoning, 30 Ounce...",[SAZONADOR TOTAL | Enhance the natural flavors...,180.00,Ounce,"Goya Foods Sazonador Total Seasoning, 30 Ounce...",goya foods sazonador total seasoning 30 ounce ...,180.00,Goya Foods
7,222007,Item Name: VineCo Original Series Chilean Sauv...,https://m.media-amazon.com/images/I/71JllaFpxM...,94.00,VineCo Original Series Chilean Sauvignon Blanc...,[Chilean Sauvignon Blanc Wine Kit - VineCo Ori...,1.00,Count,VineCo Original Series Chilean Sauvignon Blanc...,vineco original series chilean sauvignon blanc...,1.00,VineCo Original
8,37614,Item Name: NATURES PATH CEREAL FLK MULTIGRAIN ...,https://m.media-amazon.com/images/I/21O9RftI2v...,35.74,"NATURES PATH CEREAL FLK MULTIGRAIN ORG ECO, 32...",[],192.00,Fl Oz,"NATURES PATH CEREAL FLK MULTIGRAIN ORG ECO, 32...",natures path cereal flk multigrain org eco 32 ...,192.00,NATURES PATH
9,238044,Item Name: Mrs. Miller's Seedless Black Raspbe...,https://m.media-amazon.com/images/I/41miQk+RkJ...,31.80,Mrs. Miller's Seedless Black Raspberry Jam 9 O...,[Homemade Seedless Black Raspberry Jam made fr...,9.00,Ounce,Mrs. Miller's Seedless Black Raspberry Jam 9 O...,mrs miller s seedless black raspberry jam 9 ou...,9.00,Mrs
